In [ ]:
# get data
import urllib
import json
url = 'https://data.nationalgrideso.com/api/3/action/datastore_search?resource_id=bbe2cc72-a6c6-46e6-8f4e-48b879467368&limit=400'
fileobj = urllib.request.urlopen(url)
d = json.loads(fileobj.read())

print(d)


In [ ]:
# load data
import pandas as pd
results = d['result']['records']

data_df = pd.DataFrame(results)
print(len(data_df))


In [ ]:
# plot data

import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=data_df['gsp_lon'], y=data_df['gsp_lat'], mode='markers'))

fig.show()

In [ ]:
# plot on static map

import staticmaps
from staticmaps.marker import Marker


context = staticmaps.Context()

# make bottom left and top right bounding box for map, happens to be Oxford and Norwich
bottom_left = staticmaps.create_latlng(50, -8)
top_right = staticmaps.create_latlng(59, 3)

for i in range(len(data_df)-4):
    row = data_df.iloc[i]
    context.add_object(Marker(staticmaps.create_latlng(row.gnode_lat, row.gnode_lon), size=3))
    
    
# make clean map
m = context.make_clean_map_from_bounding_box(bottom_left=bottom_left, top_right=top_right, width=1000, height=1000)

m.show()
m.save('GSP.png')

In [ ]:
# plot using plotly
import numpy as np
import plotly.graph_objects as go

m = np.array(m)
print(m.shape)

x = (data_df['gnode_lon'] +8) / 11 * m.shape[1]
y = (59-data_df['gnode_lat']) / 9 * m.shape[0]

trace_map = go.Image(z=m)

layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

fig = go.Figure(data=[trace_map, go.Scatter(x=x, y=y, mode='markers', text=data_df['gnode_name'],  
                                            marker=dict(color='LightSkyBlue'))],
               layout=layout)

fig.update_yaxes(showticklabels=False)
fig.update_xaxes(showticklabels=False)



fig.show()

### This doesnt quite work, due to curitual of the world

